# The Indiana Training Program in Public and Population Health Informatics


## Exercise 2

*Competencies addressed*   

1. Apply data merging/linking and reshaping methods  (I.1.3)
2. Develop an understanding of processing different kinds of data (e.g. string processing)  (I.1.5)
3. Demonstrate the ability to describe data (find means, standard deviations, outliers, evaluate correlations etc.) (II.1.2)

*Learning objectives*  
 At the end of this module the student will be able to do the following:

1. Set up the working environment in R
2. Read the dataset prior to loading it into R
3. Import a text delimited dataset into R
4. Run summary statistics
5. Make a dataset more meaningful by linking the multiple datasets
6. Import a stata file into R
7. Understand the significance of choosing the right variable type when importing a dataset into R


This exercise uses the medication data from the Type-2 Diabetes population data file. The medication data contains detailed information on the various medications prescribed to the patients. Additional information on dosage and when the medications were prescribed is also available. We first start by loading the data into R. Once the dataset is loaded, we use `head` to view the first few rows to get an idea of what this dataset looks like.

In [2]:

 Change this path to the location where YOUR files aregetwd()  # Check the current directory.
setwd("/N/dc2/projects/T15/Sample") # Set the working folder. Change this path to the location where YOUR files are.
medication <- read.csv("medication.csv", header= TRUE)
head(medication)
summary(medication)

[1] "/N/dc2/projects/T15/Sample"

STUDYID,DAYS_MED_INDEX,DRUG_NAME,NDC_CODE,STRENGTH,DISPENSE_AMOUNT,NUMBER_OF_DAYS_SUPPLY
103,2757,GLIPIZIDE 5 MG TABLET,51079081020,NA,60,30
239,6871,NOVOLIN N,169183411,NA,-50,-55
239,811,HUMULIN N,2831501,NA,NA,NA
239,3751,HUMULIN N INJ U-100,2831501,NA,1,60
239,3819,HUMULIN R INJ U-100,2821501,NA,1,60
239,1128,HUMULIN N,2831501,NA,1,NA


    STUDYID        DAYS_MED_INDEX                        DRUG_NAME     
 Min.   :    103   Min.   :-4834   METFORMIN HCL              : 99710  
 1st Qu.: 261123   1st Qu.:  329   METFORMIN HCL 500 MG TABLET: 35672  
 Median : 513041   Median :  971   METFORMIN HCL ER           : 34704  
 Mean   : 549969   Mean   : 1302   ACTOS                      : 26168  
 3rd Qu.: 830052   3rd Qu.: 1970   GLIMEPIRIDE                : 24595  
 Max.   :1256011   Max.   : 7264   METFORMIN    TAB 500MG     : 23854  
                                   (Other)                    :465041  
    NDC_CODE            STRENGTH      DISPENSE_AMOUNT   NUMBER_OF_DAYS_SUPPLY
 Min.   :2.751e+06   Min.   :   0.0   Min.   :-473.00   Min.   :-100.00      
 1st Qu.:9.310e+07   1st Qu.:  10.0   1st Qu.:  30.00   1st Qu.:  30.00      
 Median :5.911e+08   Median : 100.0   Median :  60.00   Median :  30.00      
 Mean   :2.621e+10   Mean   : 307.8   Mean   :  69.85   Mean   :  44.06      
 3rd Qu.:6.276e+10   3rd Qu.: 500.

Now we know that this dataset has 7 variables:

* Studyid(`STUDYID`)
* Days from or to the index date (`DAYS_MED_INDEX`)
* Name of the drug (`DRUG_NAME`)
* National drug code (`NDC_CODE`)
* Strength of the medication (`STRENGTH`)
* Number of pills given out (`DISPENSE_AMOUNT`)
* Number of days supply (`NUMBER OF DAYS SUPPLY`)

The medication data is useful for examining the drugs that are being prescribed to patients and we first examine the frequency distribution of the drugs using the `table` function. <br>
The `as.data.frame` formats the table and the `head` function limits the rows to first 6 rows. 

In [3]:
head(as.data.frame(table(medication$DRUG_NAME)))

Var1,Freq
53746017810,2
ACARBOSE,279
ACARBOSE TAB 100MG,18
ACARBOSE TAB 100MGACARBOSE,3
ACARBOSE TAB 25MG,57
ACARBOSE TAB 25MGACARBOSE,27


What we find here is that given that our drugs are identified by the specific generic drug and formulation combination, we see multiple entries for the same type of drug e.g. Acarbose, Acarbose Tab 100mg, Acarbose Tab 25 mg etc. Further, even variations in spellings result in the drugs being considered distinct. This highly granular information is of little use from a research standpoint when examining most research questions. It would be helpful to aggregate these drugs into more meaningful categories, such as combining acarbose, actoplus, and all other similar drugs into the "Anti-diabetic medication" category. In order to do so, we will merge this data with a crosswalk that provides categories of drugs.

The crosswalk dataset to be used is the unindc.dta data file available in the curricular materials. 

The crosswalk was created in a different statistical software called Stata and exists as a .dta file. Thus, a different library will be used to bring the cross walk into R.

In the following cells, we will load the `haven` library, use it to read the unindc.dta data file into R, and view the first few rows.

In [7]:
# Install the haven package using following code if it is not already installed.
install.packages("haven") 
library("haven")

# Load the unindc.dta data file.
unindc <- read_dta("unindc.dta") # Read the STATA file. 
cat("Dimensions (rows columns/variables):",dim(unindc)) # Query the number of rows and columns/variables.
head(unindc) 

Warning message in install.packages("haven"):
"'lib = "/gpfs/hps/soft/rhel7/r/3.3.1/lib64/R/library"' is not writable"

ERROR: Error in install.packages("haven"): unable to install packages


The crosswalk file: unindc.dta consists of 429,473 rows and the following 9 variables:

* National Drug code (`ndc`)
* Drug name (`drug`)
* RxNorm clinical drug concept (`rxncui`)
* Concept name (`concept_name`)
* WHO/ATC code (`cscode`)
* Drug class (`drug_class`)
* Therapeutic subgroup (`sgthr`)
* Pharmacological subgroup (`sgphr`)
* Chemical subgroup (`sgchem`) 

For our purposes, we can use the various subgroups (`sgthr`-`sgchem`) to aggregate medications. Note that the `drug_class` variable, while valid for aggregation, does not provide meaningful categories.

The `ndc` variable uniquely identifies all observations in the crosswalk and can thus be used for a many-to-one merge.

### Merge
We now merge this file with our medications file using the `ndc` variable which is common to both the croswalk and the T2D medications file. However, this variable is named differently in both datasets; in the crosswalk it is called `ndc`, while in the medications dataset it is called `NDC_CODE`. Merging requires that the variable to be merged on have the same name and this is the first step in merging.

In [8]:
# Matching the NDC column name for merging. 
print("List of variables in medication data in the order they appear")
names(medication) # Query the names of the variables, output is the names of the variables in order they appear in data.
colnames(unindc)[1] <- "NDC_CODE" # Renames the first var in crosswalk to match the corresponding variable name in medications data.
print("ndc variable in crosswalk changed to match variable name in medications dataset")
names(unindc)

[1] "List of variables in medication data in the order they appear"


[1] "STUDYID"               "DAYS_MED_INDEX"        "DRUG_NAME"            
[4] "NDC_CODE"              "STRENGTH"              "DISPENSE_AMOUNT"      
[7] "NUMBER_OF_DAYS_SUPPLY"

ERROR: Error in colnames(unindc)[1] <- "NDC_CODE": object 'unindc' not found


We `merge` by matching all drug codes in the medications file with corresponding drugs in the crosswalk as follows:

In [9]:
medication_unindc <- merge(x=medication, y=unindc, by= "NDC_CODE", all.x= TRUE)
head(medication_unindc)
cat("Dimensions (rows columns/variables):",dim(medication_unindc))
summary(medication_unindc)

ERROR: Error in as.data.frame(y): object 'unindc' not found


### Match statistics

In [10]:
cat("Drugs that could not be matched:", sum(is.na(medication_unindc$drug)))  # This shows non-matching counts.  
cat("\nTotal number of unique drugs matched:",length(unique(medication_unindc$drug))) # Number of unique drugs matched. 
as.data.frame(table(medication_unindc$sgthr)) # The list of the drugs.

ERROR: Error in cat("Drugs that could not be matched:", sum(is.na(medication_unindc$drug))): object 'medication_unindc' not found


As mentioned above, only 87 unique drugs were matched, resulting in a total of 405,949 drugs being <b>not</b> matched to a corresponding drug in the crosswalk. Why does this happen?

The ndc is an 11-digit code, i.e. it consists solely of numbers. If we examine the medications file, we find that the ndc code is a string variable with leading zeroes in some codes. When we read the medications file, R tries to automatically assign variable types based on the contents of each column. Given that ndc codes are all numbers, R designates the ndc code in the medication dataset as a numeric variable dropping the leading zeros. 

Why does the crosswalk file not experience the same error?
The crosswalk file was a Stata file and includes class settings in the meta data. When the haven library imports this file, this allows R to preserve the class settings for all variables. 

In [11]:
cat("NDC class in crosswalk dataset:",class(unindc$NDC_CODE))
cat("\nNDC class in medications dataset:", class(medication$NDC_CODE))

ERROR: Error in cat("NDC class in crosswalk dataset:", class(unindc$NDC_CODE)): object 'unindc' not found


To fix the error above, we force R to read the medications file while preserving the drug codes as a string or character variable.

In [12]:
medication <- read.csv("medication.csv", header= TRUE, 
                      colClasses=c("character", "numeric", "character", "character", rep("numeric",3)) )
summary(medication)
head(medication, n= 10)

   STUDYID          DAYS_MED_INDEX   DRUG_NAME           NDC_CODE        
 Length:709744      Min.   :-4834   Length:709744      Length:709744     
 Class :character   1st Qu.:  329   Class :character   Class :character  
 Mode  :character   Median :  971   Mode  :character   Mode  :character  
                    Mean   : 1302                                        
                    3rd Qu.: 1970                                        
                    Max.   : 7264                                        
                                                                         
    STRENGTH      DISPENSE_AMOUNT   NUMBER_OF_DAYS_SUPPLY
 Min.   :   0.0   Min.   :-473.00   Min.   :-100.00      
 1st Qu.:  10.0   1st Qu.:  30.00   1st Qu.:  30.00      
 Median : 100.0   Median :  60.00   Median :  30.00      
 Mean   : 307.8   Mean   :  69.85   Mean   :  44.06      
 3rd Qu.: 500.0   3rd Qu.:  90.00   3rd Qu.:  60.00      
 Max.   :2700.0   Max.   :6000.00   Max.   : 660.00      
 N

STUDYID,DAYS_MED_INDEX,DRUG_NAME,NDC_CODE,STRENGTH,DISPENSE_AMOUNT,NUMBER_OF_DAYS_SUPPLY
103,2757,GLIPIZIDE 5 MG TABLET,51079081020,NA,60,30
239,6871,NOVOLIN N,00169183411,NA,-50,-55
239,811,HUMULIN N,00002831501,NA,NA,NA
239,3751,HUMULIN N INJ U-100,00002831501,NA,1,60
239,3819,HUMULIN R INJ U-100,00002821501,NA,1,60
239,1128,HUMULIN N,00002831501,NA,1,NA
239,1219,HUMULIN N,00002831501,NA,1,NA
239,2076,HUMULIN N INJ U-100,00002831501,NA,1,NA
239,3819,HUMULIN N INJ U-100,00002831501,NA,1,60
239,811,HUMULIN N,00002831501,NA,-1,NA


Now that we fixed the NDC code and confirm that leading zeros are preserved, we proceed with the merge as before.

In [13]:
 
medication_unindc <- merge(x= medication, y= unindc, by= "NDC_CODE", all.x= TRUE)
head(medication_unindc)
summary(medication_unindc)


ERROR: Error in as.data.frame(y): object 'unindc' not found


In [14]:
# The drugs field that did not match (NA count) is 12 now (successful matches are 720,889).
cat("Drugs that could not be matched:", sum(is.na(medication_unindc$drug)))  # This shows the non-matching count.  
cat("\nTotal number of unique drugs matched:",length(unique(medication_unindc$drug))) # Number of unique drugs matched.

ERROR: Error in cat("Drugs that could not be matched:", sum(is.na(medication_unindc$drug))): object 'medication_unindc' not found


After fixing the error, only 12 drugs are not matched compared to over 400,000 before. Also, compared to 87 unique drugs that were matched now we have 215 unique drugs that were matched.

### Let's take a look at the diabetes drugs prescribed

In [15]:
as.data.frame(table(medication_unindc$sgthr))
as.data.frame(table(medication_unindc$sgchem[medication_unindc$sgthr == "Drugs Used In Diabetes"]))
as.data.frame(table(medication_unindc$drug_class[medication_unindc$sgthr == "Drugs Used In Diabetes"]))


ERROR: Error in table(medication_unindc$sgthr): object 'medication_unindc' not found


Finally, the frequency table provides much more useful information when we use the therapeutic subgroups. How does this compare to the pharmacological subgroups and the chemical subgroup?

### Let's save the enriched medication file (in R format)
To load the file, use the following code: <br>
`load(file = "medication_unindc.RDATA")`

In [16]:
save(medication_unindc , file ="medication_unindc.RDATA")
file.info("medication_unindc.RDATA")

ERROR: Error in save(medication_unindc, file = "medication_unindc.RDATA"): object 'medication_unindc' not found


== End of Ex 1 ==